In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F
from torchinfo import summary

import sys
sys.path.append('../dgmr-coordconv')
import dgmr

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [32]:
ccs = dgmr.common.ContextConditioningStack(
    input_channels=3,
    conv_type='standard',
    output_channels=192,
)

lcs = dgmr.common.LatentConditioningStack(
    shape=(8 * 1, 128 // 32, 128 // 32),
    output_channels=192,
)

_input_channels_mult = 2
sampler = dgmr.generators.Sampler(
    forecast_steps=5,
    latent_channels=192,
    context_channels=192*_input_channels_mult,
    output_channels=1,
)

model = dgmr.generators.Generator(ccs, lcs, sampler)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model)} trainable parameters')

The model has 14493181 trainable parameters


In [34]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [35]:
x = torch.randn(3, 4, 3, 128, 128)

In [36]:
with torch.no_grad():
    o = model(x)

torch.Size([3, 48, 32, 32])
torch.Size([3, 96, 16, 16])
torch.Size([3, 192, 8, 8])
torch.Size([3, 384, 4, 4])
torch.Size([1, 192, 4, 4])


In [37]:
o.shape

torch.Size([3, 5, 1, 128, 128])

In [12]:
import pathlib
import numpy as np
p = pathlib.Path('../data/data/coords.npz')
f = np.load(p)
x_osgb = f['x_osgb']
y_osgb = f['y_osgb']


In [13]:
x_osgb.shape, y_osgb.shape

((325, 400), (325, 400))